In [2]:
from os.path import samefile
# unzip data

!unzip sample_data/cat_dog

Streaming output truncated to the last 5000 lines.
  inflating: cat_dog/training_set/training_set/dogs/dog.1154.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1155.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1158.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1161.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1166.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1171.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1174.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1180.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1185.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1186.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1187.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1189.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1196.jpg  
  inflating: cat_dog/training_set/training_set/dogs/dog.1197.jpg  
  inflating

In [ ]:
%ls

sample_data/


In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from  torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet means and stds
])
#train data
train_data = datasets.ImageFolder('cat_dog/training_set', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# test data
test_data = datasets.ImageFolder('cat_dog/test_set', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

class_names = train_data.classes

for images, labels in train_loader:
    print(images.shape)
    print(labels)
    break

torch.Size([32, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


In [5]:
cuda0 = torch.device('cuda:0')

In [6]:
class VGGNet(nn.Module):
    def __init__(self, num_classes=2):
        super(VGGNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = self.classifier(x)
        return x




In [7]:
model = VGGNet(num_classes=2).to(cuda0)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.001)
learning_rate = 0.001
batch_size = 64

def accuracy(y_hat, y):
  y_pred = torch.argmax(y_hat, dim=1)
  correct = (y_pred == y).float()
  accuracy = correct.sum()/ len(correct)
  return accuracy


In [22]:
epochs = 10
train_loss = []
train_acc = []
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0
    total_samples = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(cuda0), labels.to(cuda0)

        # Forward pass
        outputs = model(inputs)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        output = model(inputs)
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        acc = accuracy(outputs, labels)

        # Accumulate loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        running_accuracy += acc * inputs.size(0)
        total_samples += inputs.size(0)

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / total_samples
    epoch_accuracy = running_accuracy / total_samples
    train_loss.append(epoch_loss)
    train_acc.append(epoch_accuracy)
    if epoch_accuracy == 1.0 and epoch > 1:
      print(f'Epoch [{epoch+1}/{epochs}] Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
      print("*"*10, "Stop training at 100% accuracy", "*"*10)
      break
    print(f'Epoch [{epoch+1}/{epochs}] Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

Epoch [1/10] Loss: 0.0000, Accuracy: 1.0000
Epoch [2/10] Loss: 0.0000, Accuracy: 1.0000
Epoch [3/10] Loss: 0.0000, Accuracy: 1.0000
********** Stop training at 100% accuracy **********


In [23]:
print(train_acc)
print(train_loss)

[tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0')]
[0.0, 0.0, 0.0]


In [24]:
print(model)

VGGNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

In [25]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total number of parameters: {total_params:,}")
    return total_params
peram = count_parameters(model)

Total number of parameters: 128,774,530
